# Components

## Messages

In [ ]:
from langchain_core.messages import (AIMessage, HumanMessage, SystemMessage, ToolMessage)

system_message = SystemMessage("you're a good assistant, you always respond with a joke.")
human_message = HumanMessage("i wonder why it's called langchain")
ai_message = AIMessage('Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!')


## Prompt Templates

In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='Tell me a joke about cats')])

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hi!')])

## Tools

In [1]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.schema())
print(multiply.return_direct)

# Let's invoke the tool.
print(multiply.invoke({"a": 2, "b": 3}))

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'title': 'multiplySchema', 'description': 'Multiply two numbers.', 'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}
False
6


In [8]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([multiply])
query = "What is 3 * 12?"
completion = llm_with_tools.invoke(query)

multiply.invoke(completion.tool_calls[0])


ToolMessage(content='36', name='multiply', tool_call_id='call_jfRoNxhyvfPYS3hxf0b4jSvX')

In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is {a} * {b}?")
chain = prompt | llm_with_tools
completion = chain.invoke({"a": 3, "b": 12})

In [17]:
completion

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3fKQg3hNV5p9UdFUz05nx3rh', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 52, 'total_tokens': 69, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f775547e-fe9d-4e23-bafe-6a81bb371460-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_3fKQg3hNV5p9UdFUz05nx3rh', 'type': 'tool_call'}])

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableAssign

def get_tool_calls(msg):
  return msg.tool_calls[0]

my_runnable_lambda = RunnableLambda(get_tool_calls)
chain2 = my_runnable_lambda | multiply
chain2.invoke(completion)

ToolMessage(content='36', name='multiply', tool_call_id='call_3fKQg3hNV5p9UdFUz05nx3rh')